In [170]:
# !pip install crewai google-generativeai
# ! pip install langchain_google_genai
# ! pip install crewai
# ! pip install crewai google-generativeai
# ! pip install crewai[tools]
# ! pip uninstall chromadb

In [171]:
import os
from dotenv import load_dotenv
import requests
from crewai.tools import BaseTool
from crewai import Agent, Task, Crew, LLM  
# # from llama_index.llms.google_genai import GoogleGenAI 



In [172]:
load_dotenv()

True

In [173]:
gemini_llm = LLM(
    # model="gemini/gemini-2.0-flash",
    model="gemini/gemini-1.5-flash",
    # model="gemini/gemini-pro",
    temperature=0.7,
)

In [174]:
class NewsTool(BaseTool):
    name: str = 'NewsTool'
    description: str = 'Fetch and summarize latest news headlines for a topic in English.'

    def _run(self, topic: str) -> str:
        api_key = NEWS_API_KEY  # Use the NEWS_API_KEY variable loaded from .env
        url = f'https://newsapi.org/v2/everything?q={topic}&sortBy=publishedAt&apiKey={api_key}'
        # url = f'https://newsapi.org/v2/top-headlines?q={topic}&sortby=publishedAt&apiKey={api_key}'


        response = requests.get(url)
        if response.status_code != 200:
            return f'Failed to fetch news: {response.text}'
        articles = response.json().get('articles', [])
        if not articles:
            return f'No news found for topic: {topic}'
        headlines = [f"- {a['title']}" for a in articles]
        return f"Latest news about {topic}:\n" + '\n'.join(headlines)

# Test the tool
news_tool = NewsTool()

In [175]:
news_agent = Agent(
    role='News Summarizer',
    goal='Provide the latest english news for the {topic}',
    backstory='An expert news analyst who keeps up with the latest global events.',
    tools=[news_tool],
    llm=gemini_llm, 
)

In [176]:
task = Task(
    name='news_summary',
    description='Fetch and summarize latest news for the {topic} only',
    agent=news_agent,
    expected_output="Summarize the response in 3 lines only."
)

In [177]:
# create a crew with the news agent and the task
crew = Crew(
    name='News Crew',
    agents=[news_agent],
    tasks=[task],
    verbose=True
)

In [178]:
# use the crew to kickoff a task using kickoff method
# use the crew to kickoff all tasks
result = crew.kickoff(inputs={'topic': 'Artifical Intelligence'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: News Crew                                                                                                │
│  ID: eab5400d-f192-4595-a703-d9e9d039f8d0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: News Summarizer                                                                                         │
│                                                                                                                 │
│  Task: Fetch and summarize latest news for the Artifical Intelligence only                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/gitpod/.pyenv/versions/3.12.11/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: News Summarizer                                                                                         │
│                                                                                                                 │
│  Thought: tool_code                                                                                             │
│  Thought: I need to use the NewsTool to get the latest news on Artificial Intelligence.                         │
│                                                                                                                 │
│  Using Tool: NewsTool                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"topic\": \"Artificial Intelligence\"}"                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Latest news about Artificial Intelligence:                                                                     │
│  - An AI Pioneer Worries Big Tech Is Blinding Itself to New Breakthroughs                                       │
│  - That Millions of Nigerians Under Tinubu Still Die From Preventable Health Conditions Highlights Failure of   │
│  His Leadership and Empathy                                                                                     │
│  - US Judge says Google can keep Chrome, but 'exclusive deals' are a no-go                                      │
│  - Acer Expands Its Nitro Lineup with Two New AI-Ready Gaming Laptops, The Nitro V 16 And The Nitro V 16S       │
│  - Acer Launches the New Swift Air 16 – Ultralight Copilot+ PC Laptop at IFA 2025                               │
│  - At 83, this Lancaster County expert's retirement advice still rings true today                               │
│  - Is Tom Cruise the Last Action Hero?                                                                          │
│  - Trump’s Industrial Policy: What’s Right and Wrong                                                            │
│  - Why AI Narrators Will Never Be Able to Tell a Real Human Story                                               │
│  - Corebridge Financial Inc. Reduces Stake in Insight Enterprises, Inc. $NSIT                                   │
│  - Carney's plan for nation-building in batches                                                                 │
│  - Family of dead teen say ChatGPT's new parental controls not enough                                           │
│  - FinQuest 2025 (Alpha Bank) competition rewards AI applications                                               │
│  - The connected customer                                                                                       │
│  - MetLife Investment Management LLC Has $1.77 Million Stock Position in CorVel Corp. $CRVL                     │
│  - Pakistan ‘respects’ Russia’s ties with India – PM Sharif                                                     │
│  - Building the AI-enabled enterprise of the future                                                             │
│  - Google/DoJ antitrust ruling: Google must share data                                                          │
│  - The U.S. makes it harder for TSMC, SK Hynix and Samsung to produce chips in China                            │
│  - AlphaQuest LLC Has $88,000 Holdings in Innodata Inc. $INOD                                                   │
│  - BRIN underscores urgency of applying IoMT in health transformation                                           │
│  - Turku University suspends four students for cheating                                                         │
│  - CMAT, Liam Neeson, David McCullagh: Eight people who would make a great president of Ireland                 │
│  - 13 n8n Nodes : To Simplify and Supercharge Your AI Automations                                               │
│  - Australia's central bank exploring how AI can affect economy, governor says                                  │
│  - Rafferty Asset Management LLC Has $470,000 Position in SentinelOne, Inc. $S                                  │
│  - ServiceNow (NYSE:NOW) CEO Sells $5,339,369.75 in Stock                                                       │
│  - U.S. pulls TSMC’s waiver for China shipments of chip supplies                                                │
│  - Hsb...                                                                                                       │
│                                                                                                                 │
╰───────────────────────────────────────────────────────

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: News Summarizer                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Latest news about Artificial Intelligence includes advancements in AI-ready laptops, concerns about AI's       │
│  impact on jobs, and the development of new AI-powered tools and applications across various sectors.  There    │
│  are also discussions on ethical considerations, such as parental controls for AI chatbots and the potential    │
│  for AI-driven deception in job applications.  Major tech companies like Google are facing antitrust scrutiny   │
│  regarding their AI-related practices and new regulations are being discussed.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: news_summary                                                                                             │
│  Agent: News Summarizer                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: News Crew                                                                                                │
│  ID: eab5400d-f192-4595-a703-d9e9d039f8d0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Latest news about Artificial Intelligence includes advancements in AI-ready laptops, concerns    │
│  about AI's impact on jobs, and the development of new AI-powered tools and applications across various         │
│  sectors.  There are also discussions on ethical considerations, such as parental controls for AI chatbots and  │
│  the potential for AI-driven deception in job applications.  Major tech companies like Google are facing        │
│  antitrust scrutiny regarding their AI-related practices and new regulations are being discussed.               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯